# Political Labelling

In this Notebook we will asing every tweet a political affiliation based on the list of political influencer made by "La Silla Vacia". label every ReTweet as "Centro", "Derecha", "Izquierda" based on this political mapping. The we agreggate by User and asssign a POlitical Lable mased on the maximun number of Retweets this user made.

In [3]:
import pickle
import pandas as pd
import numpy as np

Load all pickle files will need

In [5]:
tweets = pd.read_pickle('../fcastrillon/Data/tweets_lite.pkl')

with open("../fcastrillon/Data/mapa.pkl", "rb") as file:
    mapa = pickle.load(file)

In [6]:
# Now we assign each RT a political label according to its influencer's label.
tweets.loc[tweets["Reference Type"] == "retweeted", "Party"] = tweets.loc[tweets["Reference Type"] == "retweeted",
                                                                         "Referenced Tweet Author ID"].map(mapa)

# We select all non-NA labeled RT.
tweets[tweets["Party"].notna()]

tweets["Party"].value_counts()

Party
Izquierda    3443228
Derecha       814456
Centro        430559
Name: count, dtype: int64

We create a 3x1 positive integer vector for every tweeter in the community that registers the number of RTs that the user has based on the political affilation. 

In [7]:
# We create lambda-functions that count the number of RTs for each political 
# label.
a = lambda x: np.sum(x == "Derecha")
b = lambda x: np.sum(x == "Izquierda")
c = lambda x: np.sum(x == "Centro")

# given per political label for each user using the lambda-functions.
rts_usuario = tweets.groupby("Author ID").agg({"Party": [a,b,c]})

rts_usuario.columns = ["Retweets Derecha", 
                       "Retweets Izquierda", 
                       "Retweets Centro"]

# Total RTs...
rts_usuario["Retweets Totales"] = rts_usuario.sum(axis=1)
# We generate dummy variables for each political label...
rts_usuario["Dummy Derecha"] = (rts_usuario["Retweets Derecha"] != 0).astype('int32')
rts_usuario["Dummy Izquierda"] = (rts_usuario["Retweets Izquierda"] != 0).astype('int32')
rts_usuario["Dummy Centro"] = (rts_usuario["Retweets Centro"] != 0).astype('int32')

# Now we determine the political affiliation by checking the index with the maximum.
rts_usuario["No Retweets"] = (rts_usuario["Retweets Totales"] == 0).astype('int32')
rts_usuario.head()

,Retweets Derecha,Retweets Izquierda,Retweets Centro,Retweets Totales,Dummy Derecha,Dummy Izquierda,Dummy Centro,No Retweets
Author ID,,,,,,,,
12996.0,1,321,109,431,1,1,1,0
777978.0,1,1,1,3,1,1,1,0
784125.0,0,46,10,56,0,1,1,0
1061601.0,0,223,3,226,0,1,1,0
1488031.0,0,0,0,0,0,0,0,1


In [8]:
# Now we determine the political affiliation by checking the index with the maximum.
rts_usuario["Afiliacion"] = rts_usuario[["Retweets Centro", 
                                         "Retweets Derecha", 
                                         "Retweets Izquierda", 
                                         "No Retweets"]].idxmax(axis=1)

rts_usuario['Afiliacion'].value_counts()

Afiliacion
Retweets Izquierda    23138
Retweets Derecha       6812
No Retweets            3815
Retweets Centro        3543
Name: count, dtype: int64

In [13]:
# Finally, we create a dictionary which stores the affiliation for each user.
user_to_party = {}

for index, row in rts_usuario.iterrows():
    author_id = int(index)
    afiliacion = row['Afiliacion']
    
    # Adding the author ID and affiliation to the dictionary
    user_to_party[author_id] = afiliacion

with open("../fcastrillon/Data/user_to_party.pkl", 'wb') as file:
    pickle.dump(user_to_party,file)

We export the Dictionary a Pickle File for Further usage

In [22]:
rts_usuario.to_pickle('../Twitter/Code/Repositorio/Data/rts_usuario.pkl')